In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/14cancermicroarraydata/14cancer_y.ytrain
/kaggle/input/14cancermicroarraydata/14cancer.info
/kaggle/input/14cancermicroarraydata/14cancer_x.xtrain


# **Statistical Learning – Data science - 2022/23 – Exercise 6 – 18/05/2023**


**Excersice 6: Analysis of Human Tumor Microarray dataset – unsupervised learning, clustering with k-means** \
Please, execute the following tasks and provide answers to the proposed questions.

* [Appendix](#exercise6)
    * [Part 1: Discover and Prepare dataset](#section1)
    * [Part 2: Cluster analysis](#section2)
        * [Prepocess data ](#subsection1)
        * [Start using K-means and then test some other clustering algorithms at your choice](#subsection2)

 ## Part 1: Discover and Prepare dataset <a class="anchor"  id="section1"></a>

1. Download the “14-cancer microarray data” from the book website https://web.stanford.edu/~hastie/ElemStatLearn/
Get information about the dataset in file 14cancer.info and in Chapter 1 (page 5) and
Chapter 14 (page 512) of the book (Hastie et al., 2009)
2. Generate a new Kernel SLDatascience_EX5_HumanTumorMicro_Clustering_Surname in Kaggle
3. Load the data in Kaggle \
• Use, for instance, the training set gene expression data \
• Load also the labels

**Introduction about DNA Expression Microarrays:**
- DNA is the basic material that makes up human chromosomes. 
- How a DNA microarray works?

The nucleotide sequences for a few thousand genes are printed on a glass slide. A target sample and a reference sample are labeled with red and green dyes, and each are hybridized with the DNA on the slide.

Through fluoroscopy, the log (red/green) intensities of RNA hybridizing at each site is measured. The result is a few thousand numbers, typically ranging from say −6 to 6, measuring the expression level of each gene in the target relative to the reference sample. Positive values indicate higher expression in the target versus the reference, and vice versa for negative values.


**Information about the dataset in file 14cancer.info**:\
A gene expression dataset collects together the expression values from a series of DNA microarray experiments, with each column representing an experiment. 

14-cancer gene expression data including: 16,063 genes, 144 training samples, 54 test samples. 
One gene per row, one sample per column
Cancer classes are labelled as follows:
1.  breast
2.  prostate
3.  lung
4.  collerectal
5.  lymphoma
6.  bladder
7.  melanoma
8.  uterus
9.  leukemia
10. renal
11. pancreas
12. ovary
13. meso
14. cns

**The challenge here is to understand how the genes and samples are organized. Typical questions include the following:**
1. which samples are most similar to each other, in terms of their expression profiles across genes?
2.  which genes are most similar to each other, in terms of their expression profiles across samples?
3.  do certain genes show very high (or low) expression for certain cancer samples?



In [2]:
# Read the trainning dataset
x_train = pd.read_csv('/kaggle/input/14cancermicroarraydata/14cancer_x.xtrain',header=None,delim_whitespace= True)
y_train = pd.read_csv('/kaggle/input/14cancermicroarraydata/14cancer_y.ytrain',header=None,delim_whitespace= True)

In [3]:
x_train.head()

,0,1,2,3,4,5,6,7,8,9,...,134,135,136,137,138,139,140,141,142,143
0,-73,-16,4,-31,-33,-37,-18,-26,-40,22,...,274,-96,-96,-124,-201,-196,34,-56,-245,-26
1,-69,-63,-45,-110,-39,-90,28,-23,-264,-14,...,-915,-221,-458,-664,-259,-369,-81,-818,-235,-1595
2,-48,-97,-112,-20,-45,-75,10,2,-335,-21,...,-303,-119,-134,-361,22,-263,-146,-1338,-127,-2085
3,13,-42,-25,-50,14,-46,30,34,18,26,...,29,243,109,21,140,162,-151,-57,197,-334
4,-86,-91,-85,-115,-56,-45,-56,-54,-163,-42,...,-171,-224,-630,-519,-277,-277,-174,-989,-562,-455


In [4]:
y_train.head()

,0,1,2,3,4,5,6,7,8,9,...,134,135,136,137,138,139,140,141,142,143
0,1,1,1,1,1,1,1,1,2,2,...,14,14,14,14,14,14,14,14,14,14


 ## Part 2: Cluster analysis <a class="anchor"  id="section2"></a>

Use the sklearn.cluster module to perform clustering analysis on the dataset. In particular, repeat the analysis proposed in section 14.3.8 of the book (Hastie et al., 2009)
*  Preprocess data, if needed (e.g., if there are missing values then remove the related columns and rows)
*  Start using K-means and then test some other clustering algorithms at your choice
*  Cluster the samples (i.e., columns). Each sample has a label (tumor type)
*  Do not use the labels in the learning phase but examine them posthoc to interpret the clusters
*  Run k-means with K from 2 to 10 and compare the clusterings in terms of within-sum of
* squares
*  Show the chart of the performance depending on K
*  Select some K and analyze the clusters producing tables such as those displayed in the book

### Prepocess data <a class="anchor"  id="subsection1"></a>

In [5]:
x_train.T.describe()

,0,1,2,3,4,5,6,7,8,9,...,16053,16054,16055,16056,16057,16058,16059,16060,16061,16062
count,144.000000,144.000000,144.000000,144.000000,144.000000,144.000000,144.000000,144.000000,144.000000,144.000000,...,144.000000,144.000000,144.000000,144.000000,144.000000,144.000000,144.000000,144.000000,144.000000,144.000000
mean,-63.645833,-257.881250,-215.511806,26.672917,-224.707639,-279.172917,-113.795139,-119.026389,-11.290278,68.313194,...,-123.929861,1271.216667,89.045833,538.594444,-3.494444,147.833333,-69.670139,21.419444,-1257.648611,-103.130556
std,65.367864,223.797476,296.444985,115.764797,164.191345,161.841157,335.428306,86.485875,67.216980,173.884559,...,104.513201,832.840702,112.680539,534.374008,104.447686,276.576271,101.915246,220.260558,698.918118,107.657973
min,-245.000000,-1595.000000,-2085.000000,-334.000000,-989.000000,-809.000000,-1621.000000,-347.000000,-413.000000,-322.000000,...,-419.000000,-67.000000,-136.000000,-49.000000,-330.000000,-311.000000,-521.000000,-361.000000,-3567.000000,-620.000000
25%,-100.250000,-312.750000,-281.750000,-26.500000,-282.750000,-381.000000,-263.750000,-176.000000,-33.250000,-16.250000,...,-183.000000,515.750000,20.250000,162.500000,-44.000000,6.750000,-107.250000,-77.250000,-1675.500000,-142.250000
50%,-55.500000,-207.500000,-119.500000,14.500000,-186.500000,-266.000000,-76.000000,-96.000000,-7.000000,41.000000,...,-112.000000,1216.000000,70.300000,446.500000,-6.000000,85.500000,-41.500000,-11.000000,-1156.000000,-66.500000
75%,-24.750000,-108.750000,-47.750000,73.000000,-107.000000,-133.000000,85.500000,-64.000000,21.000000,115.250000,...,-46.750000,2049.250000,137.500000,682.400000,27.250000,210.500000,-9.000000,42.000000,-774.000000,-35.500000
max,274.000000,28.000000,63.000000,456.000000,-10.000000,-30.000000,655.000000,196.000000,121.000000,889.000000,...,270.000000,3283.000000,751.000000,3712.000000,422.000000,1936.000000,127.000000,1754.000000,-152.000000,60.000000


In [6]:
# The overview information of x_train
x_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16063 entries, 0 to 16062
Columns: 144 entries, 0 to 143
dtypes: float64(1), int64(143)
memory usage: 17.6 MB


In [7]:
y_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Columns: 144 entries, 0 to 143
dtypes: int64(144)
memory usage: 1.2 KB


In [8]:
# Check the null values number of x_train, y_train
[sum(x_train.isnull().sum() > 1),sum(y_train.isnull().sum() > 1)]


[0, 0]

### Start using K-means and then test some other clustering algorithms at your choice <a class="anchor"  id="subsection2"></a>